In [1]:
import numpy as np
import pandas as pd 
import bokeh.io
import bokeh.plotting
import scikit_posthocs as posthoc

bokeh.io.output_notebook()

Loading BokehJS ...

# Figure 3: CAP-Mac is biased towards neurons throughout infant green monkey and newborn rhesus macaque brains.

## Figure 3b - CAP-Mac tropism in rhesus macaque

In [2]:
df_rm_quant = pd.read_csv("https://github.com/GradinaruLab/CAP-Mac/raw/main/raw-data/fig3b.csv")

# Calculate number of double positive cells expressed as percent of XFP+ cells
df_rm_quant["percent positive"] = df_rm_quant["double pos"] / df_rm_quant["reporter pos"] *100

# Calculate summary statistics
df_rm_mean = df_rm_quant.groupby(["region", "marker"])[["percent positive"]].agg(mean_percentage=("percent positive", "mean"), 
                                                                                      sem_percentage=("percent positive", "sem"), 
                                                                                      n=("percent positive", "count")).reset_index()

df_rm_quant = pd.merge(df_rm_quant, df_rm_mean, on=["region", "marker"])

######## Make some changes for plotting neatness ############
# Note: we only calculate error bars on samples that have >2 measurements. 
df_rm_quant["upper"] = df_rm_quant.apply(lambda x: (x["mean_percentage"] + x["sem_percentage"] if x["n"] > 2 else -1), axis=1) 
df_rm_quant["lower"] = df_rm_quant.apply(lambda x: (x["mean_percentage"] - x["sem_percentage"] if x["n"] > 2 else -1), axis=1) 
rng = np.random.default_rng()
df_rm_quant["jitter"] = rng.normal(0, 0.025, len(df_rm_quant)) 
df_rm_quant["cats"] = df_rm_quant.apply(lambda x: (x["region"], x["jitter"]), axis=1)
######## Make some changes for plotting neatness ############

df_rm_quant.head()

,file,variant,animal id,region,marker,reporter pos,double pos,marker pos,percent positive,mean_percentage,sem_percentage,n,upper,lower,jitter,cats
0,49845_neun12_par-ctx_10x_2022-09-06_1,CAP-Mac,RM-009,Parietal lobe,neun,61,48,6076,78.688525,60.370285,2.726564,18,63.096849,57.643721,-0.017438,"(Parietal lobe, -0.017438061358966816)"
1,49845_neun12_par-ctx_10x_2022-09-06_2,CAP-Mac,RM-009,Parietal lobe,neun,66,46,9039,69.696970,60.370285,2.726564,18,63.096849,57.643721,0.017324,"(Parietal lobe, 0.017323984175332093)"
2,49845_neun12_par-ctx_10x_2022-09-06_3,CAP-Mac,RM-009,Parietal lobe,neun,27,19,10728,70.370370,60.370285,2.726564,18,63.096849,57.643721,0.065026,"(Parietal lobe, 0.06502620340054599)"
3,49845_neun12_par-ctx_10x_2022-09-06_4,CAP-Mac,RM-009,Parietal lobe,neun,70,43,12340,61.428571,60.370285,2.726564,18,63.096849,57.643721,-0.012442,"(Parietal lobe, -0.012441951363203849)"
4,49845_neun12_par-ctx_10x_2022_07_20__11_52_37,CAP-Mac,RM-009,Parietal lobe,neun,102,63,8388,61.764706,60.370285,2.726564,18,63.096849,57.643721,0.007497,"(Parietal lobe, 0.007496612003604646)"


In [3]:
############### BOKEH FIGURE SETTINGS ###################
x_range_brain = ["Frontal lobe", "Parietal lobe", "Temporal lobe", "Caudate" , "Putamen", "Thalamus"]
stain_color = bokeh.transform.factor_cmap("marker", palette=bokeh.palettes.Pastel1[3], factors=["s100", "neun"])
animal_id = ["RM-009", "RM-010"]
markers = ["square", "diamond"] 
marker_color=list(bokeh.palettes.Colorblind[8][-2:])

figure_width = 500
figure_height = 300
width = 0.35
dodge = width/2

marker_size = 5
error_size=10
error_line_width=0.5

p = bokeh.plotting.figure(x_range=x_range_brain, height=figure_height, width=figure_width, title="CAP-Mac tropism in rhesus macaque")
p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=45
p.axis.major_tick_in = 0
p.axis.major_label_text_color = "#000000"
p.axis.major_label_text_font_size="12pt"
p.axis.axis_label_text_align = "right"
p.add_layout(bokeh.models.Legend(), "right")
############### BOKEH FIGURE SETTINGS ###################

In [4]:
# Add bar plots
p.vbar(source=df_rm_quant.loc[df_rm_quant["marker"]=="neun"], x=bokeh.transform.dodge("region", -dodge, range=p.x_range), 
       top="mean_percentage", width=width, fill_color=stain_color, line_color="black", legend_label="NeuN")
p.vbar(source=df_rm_quant.loc[df_rm_quant["marker"]=="s100"], x=bokeh.transform.dodge("region", dodge, range=p.x_range), 
       top="mean_percentage", width=width, fill_color=stain_color, line_color="black", legend_label="S100β")

# Add markers overlay
p.scatter(x=bokeh.transform.dodge("cats", dodge, range=p.x_range), y="percent positive", 
          source=df_rm_quant.loc[df_rm_quant["marker"]=="s100"], marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_color, animal_id), line_color="black", line_width=0.25, legend_group="animal id")

p.scatter(x=bokeh.transform.dodge("cats", -dodge, range=p.x_range), y="percent positive", 
          source=df_rm_quant.loc[df_rm_quant["marker"]=="neun"], marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_color, animal_id), line_color="black", line_width=0.25)

############### ADD ERROR BARS ###################
source_error_s100 = bokeh.models.ColumnDataSource(data=df_rm_quant.loc[df_rm_quant["marker"]=="s100"])
source_error_neun = bokeh.models.ColumnDataSource(data=df_rm_quant.loc[df_rm_quant["marker"]=="neun"])

w_s100 = bokeh.models.Whisker(source=source_error_s100, base=bokeh.transform.dodge("region", dodge, range=p.x_range), 
                              upper="upper", lower="lower", level="overlay", line_width=error_line_width)
w_neun = bokeh.models.Whisker(source=source_error_neun, base=bokeh.transform.dodge("region", -dodge, range=p.x_range), 
                            upper="upper", lower="lower", level="overlay", line_width=error_line_width)

w_s100.upper_head.line_width=error_line_width
w_s100.upper_head.size=error_size
w_s100.lower_head.line_width=error_line_width
w_s100.lower_head.size=error_size

w_neun.upper_head.line_width=error_line_width
w_neun.upper_head.size=error_size
w_neun.lower_head.line_width=error_line_width
w_neun.lower_head.size=error_size

p.add_layout(w_s100)
p.add_layout(w_neun)
############### ADD ERROR BARS ###################

p.y_range = bokeh.models.Range1d(0, 110)
bokeh.io.show(p)

### Total coverage of CAP-Mac in rhesus macaque

In [5]:
df_coverage=df_rm_quant.groupby(["marker"]).sum(numeric_only=True)
df_coverage["coverage"] = df_coverage["double pos"] / df_coverage["marker pos"] * 100

print("%0.2f%% of all NeuN+ cells sampled are XFP+." % (df_coverage.loc["neun", "coverage"]))
print("%0.2f%% of all S100β+ cells sampled are XFP+." % (df_coverage.loc["s100", "coverage"]))

1.12% of all NeuN+ cells sampled are XFP+.
0.04% of all S100β+ cells sampled are XFP+.


## Figure 3d - CAP-Mac tropism in green monkey

In [6]:
df_gm_quant_cap_mac = pd.read_csv("https://github.com/GradinaruLab/CAP-Mac/raw/main/raw-data/fig3d.csv")

# Calculate number of double positive cells expressed as percent of GFP+ cells
df_gm_quant_cap_mac["percent positive"] = df_gm_quant_cap_mac["double pos"] / df_gm_quant_cap_mac["reporter pos"] *100

# Calculate summary statistics
df_mean = df_gm_quant_cap_mac.groupby(["region", "variant", "marker"])[["percent positive"]].agg(mean_percentage=("percent positive", "mean"), 
                                                                                              std_percentage=("percent positive", "std"), 
                                                                                              n=("percent positive", "count")).reset_index()

df_gm_quant_cap_mac = pd.merge(df_gm_quant_cap_mac, df_mean, on=["region", "variant", "marker"])
df_gm_quant_cap_mac["error"] = df_gm_quant_cap_mac["std_percentage"] / df_gm_quant_cap_mac["n"].apply(np.sqrt)

######## Make some changes for plotting neatness ############
# Note: we only calculate error bars on samples that have >2 measurements. 
df_gm_quant_cap_mac["upper"] = df_gm_quant_cap_mac.apply(lambda x: (x["mean_percentage"] + x["error"] if x["n"] > 2 else -1), axis=1) 
df_gm_quant_cap_mac["lower"] = df_gm_quant_cap_mac.apply(lambda x: (x["mean_percentage"] - x["error"] if x["n"] > 2 else -1), axis=1) 
rng = np.random.default_rng()
df_gm_quant_cap_mac["jitter"] = rng.normal(0, 0.05, len(df_gm_quant_cap_mac)) 
df_gm_quant_cap_mac["cats"] = df_gm_quant_cap_mac.apply(lambda x: (x["region"], x["jitter"]), axis=1)
######## Make some changes for plotting neatness ############

df_gm_quant_cap_mac.head()

,file,variant,animal id,region,marker,reporter pos,double pos,marker pos,percent positive,mean_percentage,std_percentage,n,error,upper,lower,jitter,cats
0,C010.7.2a_Cd,CAP-Mac,C010,Caudate,neun,40,24,1044,60.000000,43.699656,17.457563,11,5.263653,48.963309,38.436003,-0.002698,"(Caudate, -0.0026980973397874777)"
1,C010.7.2b_Cd,CAP-Mac,C010,Caudate,neun,41,21,1074,51.219512,43.699656,17.457563,11,5.263653,48.963309,38.436003,-0.047785,"(Caudate, -0.04778453361596166)"
2,C010.7.3a_Cd,CAP-Mac,C010,Caudate,neun,57,29,987,50.877193,43.699656,17.457563,11,5.263653,48.963309,38.436003,-0.043003,"(Caudate, -0.04300283101326338)"
3,C010.7.3b_Cd,CAP-Mac,C010,Caudate,neun,48,20,1004,41.666667,43.699656,17.457563,11,5.263653,48.963309,38.436003,-0.021662,"(Caudate, -0.02166200543719491)"
4,C010.7.4a_Cd,CAP-Mac,C010,Caudate,neun,44,13,1097,29.545455,43.699656,17.457563,11,5.263653,48.963309,38.436003,-0.061654,"(Caudate, -0.06165382351448612)"


In [7]:
############### BOKEH FIGURE SETTINGS ###################
animal_id = ["C002", "C016", "C017", "C010"]
markers = ["circle", "triangle", "square", "diamond"] 
marker_color=list(bokeh.palettes.Colorblind[6][2:])

figure_width = 500
figure_height = 300
width = 0.35
dodge = width/2

marker_size = 5
error_size=10
error_line_width=0.5

p = bokeh.plotting.figure(x_range=x_range_brain, height=figure_height, width=figure_width, title="CAP-Mac tropism in green monkey")

p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=45
p.axis.major_tick_in = 0
p.axis.major_label_text_color = "#000000"
p.axis.major_label_text_font_size="12pt"
p.axis.axis_label_text_align = "right"
p.add_layout(bokeh.models.Legend(), "right")
############### BOKEH FIGURE SETTINGS ###################

In [8]:
# Add bar and scatter plots
p.vbar(source=df_gm_quant_cap_mac.loc[df_gm_quant_cap_mac["marker"]=="neun"], x=bokeh.transform.dodge("region", -dodge, range=p.x_range), 
       top="mean_percentage", width=width, fill_color=stain_color, line_color="black", legend_label="NeuN")
p.vbar(source=df_gm_quant_cap_mac.loc[df_gm_quant_cap_mac["marker"]=="s100"], x=bokeh.transform.dodge("region", dodge, range=p.x_range), 
       top="mean_percentage", width=width, fill_color=stain_color, line_color="black", legend_label="s100β")

p.scatter(x=bokeh.transform.dodge("cats", dodge, range=p.x_range), y="percent positive", 
          source=df_gm_quant_cap_mac.loc[df_gm_quant_cap_mac["marker"]=="s100"], marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_color, animal_id), line_color="black", line_width=0.25, legend_group="animal id")

p.scatter(x=bokeh.transform.dodge("cats", -dodge, range=p.x_range), y="percent positive", 
          source=df_gm_quant_cap_mac.loc[df_gm_quant_cap_mac["marker"]=="neun"], marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_color, animal_id), line_color="black", line_width=0.25)

############### ADD ERROR BARS ###################
source_error_s100 = bokeh.models.ColumnDataSource(data=df_gm_quant_cap_mac.loc[df_gm_quant_cap_mac["marker"]=="s100"])
source_error_neun = bokeh.models.ColumnDataSource(data=df_gm_quant_cap_mac.loc[df_gm_quant_cap_mac["marker"]=="neun"])

w_s100 = bokeh.models.Whisker(source=source_error_s100, base=bokeh.transform.dodge("region", dodge, range=p.x_range), 
                              upper="upper", lower="lower", level="overlay", line_width=error_line_width)
w_neun = bokeh.models.Whisker(source=source_error_neun, base=bokeh.transform.dodge("region", -dodge, range=p.x_range), 
                            upper="upper", lower="lower", level="overlay", line_width=error_line_width)

w_s100.upper_head.line_width=error_line_width
w_s100.upper_head.size=error_size
w_s100.lower_head.line_width=error_line_width
w_s100.lower_head.size=error_size

w_neun.upper_head.line_width=error_line_width
w_neun.upper_head.size=error_size
w_neun.lower_head.line_width=error_line_width
w_neun.lower_head.size=error_size

p.add_layout(w_s100)
p.add_layout(w_neun)
############### ADD ERROR BARS ###################

p.y_range = bokeh.models.Range1d(0, 110)
bokeh.io.show(p)

### Total coverage of CAP-Mac in green monkey

In [9]:
df_coverage=df_gm_quant_cap_mac.groupby("marker").sum(numeric_only=True)
df_coverage["coverage"] = df_coverage["double pos"] / df_coverage["marker pos"] * 100
print("%0.2f%% of all NeuN+ cells sampled are GFP+." % (df_coverage.loc["neun", "coverage"]))
print("%0.2f%% of all S100β+ cells sampled are GFP+." % (df_coverage.loc["s100", "coverage"]))

1.30% of all NeuN+ cells sampled are GFP+.
0.64% of all S100β+ cells sampled are GFP+.


## Figure 3e - AAV9 tropism in green monkey

In [10]:
df_gm_quant_aav9 = pd.read_csv("https://github.com/GradinaruLab/CAP-Mac/raw/main/raw-data/fig3e.csv")

# Calculate number of double positive cells expressed as percent of GFP+ cells
df_gm_quant_aav9["percent positive"] = df_gm_quant_aav9["double pos"] / df_gm_quant_aav9["reporter pos"] *100

# Calculate summary statistics
df_mean = df_gm_quant_aav9.groupby(["region", "variant", "marker"])[["percent positive"]].agg(mean_percentage=("percent positive", "mean"),
                                                                                              std_percentage=("percent positive", "std"),
                                                                                              n=("percent positive", "count")).reset_index()
df_gm_quant_aav9 = pd.merge(df_gm_quant_aav9, df_mean, on=["region", "variant", "marker"])
df_gm_quant_aav9["error"] = df_gm_quant_aav9["std_percentage"] / df_gm_quant_aav9["n"].apply(np.sqrt)

######## Make some changes for plotting neatness ############
# Note: we only calculate error bars on samples that have >2 measurements. 
df_gm_quant_aav9["upper"] = df_gm_quant_aav9.apply(lambda x: (x["mean_percentage"] + x["error"] if x["n"] > 2 else -1), axis=1) 
df_gm_quant_aav9["lower"] = df_gm_quant_aav9.apply(lambda x: (x["mean_percentage"] - x["error"] if x["n"] > 2 else -1), axis=1) 
rng = np.random.default_rng()
df_gm_quant_aav9["jitter"] = rng.normal(0, 0.05, len(df_gm_quant_aav9)) 
df_gm_quant_aav9["cats"] = df_gm_quant_aav9.apply(lambda x: (x["region"], x["jitter"]), axis=1)
######## Make some changes for plotting neatness ############

df_gm_quant_aav9.head()

,file,variant,animal id,region,marker,reporter pos,double pos,marker pos,percent positive,mean_percentage,std_percentage,n,error,upper,lower,jitter,cats
0,C002.7.14a_Cd,AAV9,C002,Caudate,neun,18,7,874,38.888889,29.925554,6.953591,9,2.317864,32.243418,27.60769,0.045536,"(Caudate, 0.045535543765158365)"
1,C002.7.15a_Cd,AAV9,C002,Caudate,neun,26,4,927,15.384615,29.925554,6.953591,9,2.317864,32.243418,27.60769,0.041917,"(Caudate, 0.04191671633729321)"
2,C002.7.16a_Cd,AAV9,C002,Caudate,neun,16,5,2678,31.250000,29.925554,6.953591,9,2.317864,32.243418,27.60769,-0.000616,"(Caudate, -0.0006157681821472107)"
3,C016.7.2a_Cd,AAV9,C016,Caudate,neun,51,14,783,27.450980,29.925554,6.953591,9,2.317864,32.243418,27.60769,-0.029276,"(Caudate, -0.029275708610864648)"
4,C016.7.2b_Cd,AAV9,C016,Caudate,neun,43,14,873,32.558140,29.925554,6.953591,9,2.317864,32.243418,27.60769,-0.026982,"(Caudate, -0.026982262960665377)"


In [11]:
############### BOKEH FIGURE SETTINGS ###################
animal_id = ["C002", "C016", "C017", "C010"]
markers = ["circle", "triangle", "square", "diamond"] 
marker_color=list(bokeh.palettes.Colorblind[6][2:])

figure_width = 500
figure_height = 300
width = 0.35
dodge = width/2

marker_size = 5
error_size=10
error_line_width=0.5

p = bokeh.plotting.figure(x_range=x_range_brain, height=figure_height, width=figure_width, title="AAV9 tropism in green monkey")

p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=45
p.axis.major_tick_in = 0
p.axis.major_label_text_color = "#000000"
p.axis.major_label_text_font_size="12pt"
p.axis.axis_label_text_align = "right"
p.add_layout(bokeh.models.Legend(), "right")
############### BOKEH FIGURE SETTINGS ###################

In [12]:
# Add bar and scatter plots
p.vbar(source=df_gm_quant_aav9.loc[df_gm_quant_aav9["marker"]=="neun"], x=bokeh.transform.dodge("region", -dodge, range=p.x_range), 
       top="mean_percentage", width=width, fill_color=stain_color, line_color="black", legend_label="NeuN")
p.vbar(source=df_gm_quant_aav9.loc[df_gm_quant_aav9["marker"]=="s100"], x=bokeh.transform.dodge("region", dodge, range=p.x_range), 
       top="mean_percentage", width=width, fill_color=stain_color, line_color="black", legend_label="s100β")

p.scatter(x=bokeh.transform.dodge("cats", dodge, range=p.x_range), y="percent positive", 
          source=df_gm_quant_aav9.loc[df_gm_quant_aav9["marker"]=="s100"], marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_color, animal_id), line_color="black", line_width=0.25, legend_group="animal id")

p.scatter(x=bokeh.transform.dodge("cats", -dodge, range=p.x_range), y="percent positive", 
          source=df_gm_quant_aav9.loc[df_gm_quant_aav9["marker"]=="neun"], marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_color, animal_id), line_color="black", line_width=0.25)

############### ADD ERROR BARS ###################
source_error_s100 = bokeh.models.ColumnDataSource(data=df_gm_quant_aav9.loc[df_gm_quant_aav9["marker"]=="s100"])
source_error_neun = bokeh.models.ColumnDataSource(data=df_gm_quant_aav9.loc[df_gm_quant_aav9["marker"]=="neun"])

w_s100 = bokeh.models.Whisker(source=source_error_s100, base=bokeh.transform.dodge("region", dodge, range=p.x_range), 
                              upper="upper", lower="lower", level="overlay", line_width=error_line_width)
w_neun = bokeh.models.Whisker(source=source_error_neun, base=bokeh.transform.dodge("region", -dodge, range=p.x_range), 
                            upper="upper", lower="lower", level="overlay", line_width=error_line_width)

w_s100.upper_head.line_width=error_line_width
w_s100.upper_head.size=error_size
w_s100.lower_head.line_width=error_line_width
w_s100.lower_head.size=error_size

w_neun.upper_head.line_width=error_line_width
w_neun.upper_head.size=error_size
w_neun.lower_head.line_width=error_line_width
w_neun.lower_head.size=error_size

p.add_layout(w_s100)
p.add_layout(w_neun)
############### ADD ERROR BARS ###################

p.y_range = bokeh.models.Range1d(0, 110)
bokeh.io.show(p)

### Total coverage of AAV9 in green monkey

In [13]:
df_coverage=df_gm_quant_aav9.groupby("marker").sum(numeric_only=True)
df_coverage["coverage"] = df_coverage["double pos"] / df_coverage["marker pos"] * 100
print("%0.2f%% of all NeuN+ cells sampled are GFP+." % (df_coverage.loc["neun", "coverage"]))
print("%0.2f%% of all S100β+ cells sampled are GFP+." % (df_coverage.loc["s100", "coverage"]))

0.49% of all NeuN+ cells sampled are GFP+.
1.86% of all S100β+ cells sampled are GFP+.


## Figure 3f - Brainwide eGFP transgene DNA distribution in green monkeys

In [14]:
df_agm_biod = pd.read_csv("https://github.com/GradinaruLab/CAP-Mac/raw/main/raw-data/fig3f.csv")

# Calculate summary statistics
df_agm_mean = df_agm_biod.groupby(["region", "variant"]).agg(mean_moi=("moi", "mean"), mean_vg=("vg/ug dna", "mean")).reset_index()
df_agm_biod = pd.merge(df_agm_biod, df_agm_mean, on=["region","variant"])

######## Make some changes for plotting neatness ############
rng = np.random.default_rng()
df_agm_biod["jitter"] = rng.normal(0, 0.025, len(df_agm_biod)) 
df_agm_biod["cats"] = df_agm_biod.apply(lambda x: (x["region"], x["jitter"]), axis=1)
######## Make some changes for plotting neatness ############

df_agm_biod.head()

,variant,animal id,tissue,region,moi,vg/ug dna,mean_moi,mean_vg,jitter,cats
0,AAV9,C002,Brain,Caudate,0.674905,112439.21940,0.487271,81179.411140,-0.001926,"(Caudate, -0.0019257079424354923)"
1,AAV9,C016,Brain,Caudate,0.299637,49919.60288,0.487271,81179.411140,-0.077659,"(Caudate, -0.07765931068272598)"
2,AAV9,C016,Brain,Cerebellum,0.085913,14313.10278,0.075749,12619.748875,0.037113,"(Cerebellum, 0.03711332729705379)"
3,AAV9,C002,Brain,Cerebellum,0.065585,10926.39497,0.075749,12619.748875,0.060100,"(Cerebellum, 0.06010000250672022)"
4,AAV9,C002,Brain,Frontal lobe,0.915763,152566.14920,0.555924,92616.895735,0.036361,"(Frontal lobe, 0.03636065716991991)"


In [15]:
############### BOKEH FIGURE SETTINGS ###################
brain = ["Frontal lobe", "Parietal lobe", "Temporal lobe", "Occipital lobe", 
         "Hippocampus", "Caudate" , "Putamen", "Thalamus" , "Hypothalamus",
         "Medulla","Cerebellum"]
variant_color = bokeh.transform.factor_cmap("variant", palette=["gray", "white"], factors=["AAV9", "CAP-Mac"])

animal_id = ["C002", "C016", "C017", "C010"]
markers = ["circle", "triangle", "square", "diamond"] 
marker_color=list(bokeh.palettes.Colorblind[6][2:])

figure_width = 1000
figure_height = 300
width = 0.35
dodge = width/2

marker_size = 10
error_size=4
error_line_width=0.5

p = bokeh.plotting.figure(x_range=brain, height=figure_height, width=figure_width, title="Brainwide eGFP transgene DNA distribution in green monkey")

p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=45
p.axis.major_tick_in = 0
p.axis.major_label_text_color = "#000000"
p.axis.major_label_text_font_size="12pt"
p.axis.axis_label_text_align = "right"
p.add_layout(bokeh.models.Legend(), "right")
############### BOKEH FIGURE SETTINGS ###################

In [16]:
p.vbar(source=df_agm_biod.loc[df_agm_biod["variant"]=="AAV9"], x=bokeh.transform.dodge("region", -dodge, range=p.x_range), 
       top="mean_vg", width=width, fill_color=variant_color, line_color="black", legend_label="AAV9")
p.vbar(source=df_agm_biod.loc[df_agm_biod["variant"]=="CAP-Mac"], x=bokeh.transform.dodge("region", dodge, range=p.x_range), 
       top="mean_vg", width=width, fill_color=variant_color, line_color="black", legend_label="CAP-Mac")

p.scatter(x=bokeh.transform.dodge("cats", -dodge, range=p.x_range), y="vg/ug dna", 
          source=df_agm_biod.loc[df_agm_biod["variant"]=="AAV9"], marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_color, animal_id), line_color="black", legend_group="animal id")

p.scatter(x=bokeh.transform.dodge("cats", dodge, range=p.x_range), y="vg/ug dna", 
          source=df_agm_biod.loc[df_agm_biod["variant"]=="CAP-Mac"], marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_color, animal_id), line_color="black",  legend_group="animal id")


p.y_range = bokeh.models.Range1d(0, np.ceil(df_agm_biod["vg/ug dna"].max()*1.1))
bokeh.io.show(p)